In [1]:
import pymongo
import sys
import json
import time
import urllib.parse
from datetime import datetime
from pymongo import MongoClient

import itertools
from itertools import groupby
from operator import itemgetter

#Importing other python files
import import_ipynb
import Integrated_Code_1
from Integrated_Code_1 import *
import Automated_DataPrep
from Automated_DataPrep import *

import schedule 
import time 

client = MongoClient('localhost', 27017) #Establishing connection with MongoDB 
client.admin.authenticate('DasSystem', 'DasSystem',mechanism ='SCRAM-SHA-1', source='admin')
db = client['DASMongoDB'] #Database name
collection = db['DAS.Grob21.Package_2'] #Collection name to get the ToolID
    
def Restart(Last_tool_timestamp):
    local_Last_tool_timestamp = Last_tool_timestamp;
    
    ##### SPLITTING FRAMES #####
    #Initialize the table_values with the table name which you want to split frame data 
    table_values = [
        {
            "tab_name":"Prediction_Spindle1",
            "column_name":"TrendingSp1",
            "sub_tables":["TrendingSp1_Value1","TrendingSP1_Value2","TrendingSp1__Value3","TrendingSp1_Value4"] ## any table name to create in mongoDB
        },
        {
            "tab_name":"Prediction_Spindle2",
            "column_name":"TrendSp2",
            "sub_tables":["TrendingSp2_Value1","TrendingSp2_Value2","TrendingSp2__Value3","TrendingSp2_Value4"] ## any table name to create in mongoDB
        }
     ]

    #Function to create folders and file for each Tool
    def Time(t):
        t_year = t['timeStamp']['year']
        t_month = t['timeStamp']['month']
        t_day = t['timeStamp']['day']
        t_hour = t['timeStamp']['hour']
        t_minute = t['timeStamp']['minute']
        t_second = t['timeStamp']['second']
        t_ms = t['timeStamp']['ms']
        t_us = t['timeStamp']['us']
        x_1 = ('{0}-{1}-{2} {3}:{4}:{5}.{6}'.format(t_year,t_month, t_day, t_hour, t_minute, t_second, t_ms))
        return x_1

    def getDataFrame(frameData, dateObj,files):
        #Variables for creating tables
        timestampStr = dateObj.strftime("%Y-%m-%d %H:%M:%S.%f")
        Value1ToAdd = { "DateTime": timestampStr, "DataItem": [] }
        Value2ToAdd = { "DateTime": timestampStr, "DataItem": [] }
        Value3ToAdd = { "DateTime": timestampStr, "DataItem": [] }
        Value4ToAdd = { "DateTime": timestampStr, "DataItem": [] }
        for val in frameData:
            timeJson = Time(val)
            for idx, oneData in enumerate(val['data']):
                data1 ={ "TimeStamp":"", "Data":"" }
                data2 ={ "TimeStamp":"", "Data":"" }
                data3 ={ "TimeStamp":"", "Data":"" }
                data4 ={ "TimeStamp":"", "Data":"" }
                
                #Checking data array in frames
                if idx == 0:
                    data1["TimeStamp"]=timeJson
                    data1["Data"]=oneData
                    Value1ToAdd["DataItem"].append(data1)
                elif idx == 1:
                    data2["TimeStamp"]=timeJson
                    data2["Data"]=oneData
                    Value2ToAdd["DataItem"].append(data2)
                elif idx == 2:
                    data3["TimeStamp"]=timeJson
                    data3["Data"]=oneData 
                    Value3ToAdd["DataItem"].append(data3)
                else:
                    data4["TimeStamp"]=timeJson
                    data4["Data"]=oneData 
                    Value4ToAdd["DataItem"].append(data4)

        #Creating new tables for each values in data array   
        if Value1ToAdd is not 0:
            db[files[0]].insert_one(Value1ToAdd)
        if Value2ToAdd is not 0:
            db[files[1]].insert_one(Value2ToAdd)
        if Value3ToAdd is not 0:
            db[files[2]].insert_one(Value3ToAdd)
        if Value4ToAdd is not 0:
            db[files[3]].insert_one(Value4ToAdd)

    ##### START SPLITTING IF THE COLLECTION WITHIN RANGE #####   
    def start(range1, range2):

        #Loop runs upto each object in the collection
        for table in table_values:
            table_name = table["tab_name"]
            tab = db[table_name].find()
            objCount = 0
            for val in tab:
                if (range1 <= val['DateTime'] <= range2):    
                    try:
                        #Getting json data from data string  
                        objCount +=1
                        j_Data= json.loads(val[table["column_name"]])
                        frameData = j_Data["frames"]
                        if len(frameData)is not 0: 
                            getDataFrame(frameData,val["DateTime"],table["sub_tables"])
                    except Exception  as e:
                        #Identifying the missing datas in DB collection
                        print("May be missing a data in Db collection object no:", objCount, file=sys.stderr)     


    completeSet = []; #Stores the collection records
    currentSet = []; #Groups in terms of workpieces
    completeSubset = []; #Lists only those records between the two tools

    ## Storing the complete collection records in a variable
    for x in collection.find():
      if (x['DateTime'] > Last_tool_timestamp): #Start splitting data from the last accessed record 
                                                #in the already existing collection
        completeSet.append(x);

    ## Grouping in terms of the workpiece numbers (Makes sets)
    sorted_completeSet = sorted(completeSet, key=itemgetter('Workpiece_1'))
    currentSet= [list(group) for key, group in itertools.groupby(completeSet, key=lambda x: x['Workpiece_1'])]

    ##Filtering records between two tools
    tool1 = 17010;
    tool2 = 17050;

    for i in range(0,len(currentSet)):
        for j in range(0,len(currentSet[i])):
          for k in range(j+1,len(currentSet[i])):
            if (currentSet[i][j]['Tool_ID'] == tool1 and currentSet[i][k]['Tool_ID'] == tool2):
             completeSubset.append([dict(currentSet[i][x])for x in range(j,k+1)])
    
    if len(completeSubset) == 0:
     time.sleep(480)
     Restart(local_Last_tool_timestamp);        

    ##Splitting the frames for each group of workpiece_numbers            
    for i in range(0,len(completeSubset)): 
        start(completeSubset[i][0]['DateTime'], completeSubset[i][len(completeSubset[i])-1]['DateTime'])
        for j in range(0,len(completeSubset[i])):
            current_tool_timestamp = completeSubset[i][j]['DateTime'];

        if (((i+1) == len(completeSubset)) and ((j+1) == len(completeSubset[i]))):
            Last_folder = Integrated_Code_1.start(completeSubset[0][0]['DateTime'],completeSubset[i][j]['DateTime'])
            Automated_DataPrep.PrepareData(Last_folder);
            Restart(current_tool_timestamp);



importing Jupyter notebook from Integrated_Code_1.ipynb
